# Create Agent with Function Definition

In this notebook we will create an Agent for Amazon Bedrock using the new capabilities for function definition.

We will use an HR agent as example. With this agent, you can check your available vacation days and request a new vacation leave. We will use an AWS Lambda function to define the logic that checks for the number of available vacation days and confirm new time off.

For this example, we will manage employee data in an [SQLite](https://www.sqlite.org/) database and generate synthetic data for demonstrating the agent.

## Prerequisites
Before starting, let's update the botocore and boto3 packages to ensure we have the latest version

In [ ]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

Let's now check the boto3 version to ensure the correct version has been installed. Your version should be greater than or equal to 1.34.90.

In [ ]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

In [ ]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Let's now create the boto3 clients for the required AWS services

In [ ]:
# getting boto3 clients for required AWS services
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

Next we can set some configuration variables for the agent and for the lambda function being created

## Preparing Agent

Let's create a DRAFT version of the agent that can be used for internal testing.


In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

## Invoke Agent

Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [ ]:
def simple_agent_invoke(input_text):
    session_id:str = str(uuid.uuid1())
    agent_id = "2EOQS4ZE93"
    agent_alias_id = "TSTALIASID"    
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=True, 
        endSession= False
    )
    logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                a = event['trace']
                try:
                    a['trace']['preProcessingTrace']['modelInvocationInput']['text'] = "TEXT"
                except KeyError:
                    pass          
                try:
                    a['trace']['orchestrationTrace']['modelInvocationInput']['text'] = "TEXT"
                except KeyError:
                    pass
                logger.info(json.dumps(a, indent=2))
                logger.info("\n=====================================================================================\n")
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [ ]:
simple_agent_invoke("rdspg-storage RDS instance is running out of storage space. Fix the issue using runbook")

In [ ]:
simple_agent_invoke("rdspg-iops RDS instance is reaching IOPS limit. Fix the issue using runbook")

In [ ]:
simple_agent_invoke("rdspg-cpu-prod RDS instance is encountering high CPU usage. Fix the issue using runbook")